**Professor:** Wesley Vieira
**6º Período Noite**

# Algoritmos de otimização - calendário de voos

> **Tarefas para as equipes:** implementar o algoritmo **Simulated Annealing**, **Algoritmo Genético** e ***Hill Climb*** e comparar os resultados.

> **Apresentar um relatório com os resultados das simulações**. O relatório que será utilizado como métrica para avaliação.


## Representação do problema

In [1]:
pessoas = [('Lisboa', 'LIS'),
           ('Madrid', 'MAD'),
           ('Paris', 'CDG'),
           ('Dublin', 'DUB'),
           ('Bruxelas', 'BRU'),
           ('Londres', 'LHR')]
custo_espera_por_hora = 10.0

In [2]:
destino = 'FCO'

In [ ]:
voos = {('BRU', 'FCO'): ['15:44', '18:55', 382]}

In [3]:
voos = {}

for linha in open('/content/flights.txt'):

    # O resultado será: ['REC', 'GRU', '07:30', '10:00', '500']
    dados = linha.strip().split(',')

    origem        = dados[0]
    destino       = dados[1]
    saida_texto   = dados[2]
    chegada_texto = dados[3]
    preco_texto   = dados[4]

    # Convertendo o horário de texto para inteiro

    partes_saida = saida_texto.split(':')
    horas_saida = int(partes_saida[0])
    minutos_saida = int(partes_saida[1])
    saida_numero = horas_saida + minutos_saida / 60

    partes_chegada = chegada_texto.split(':')
    horas_chegada = int(partes_chegada[0])
    minutos_chegada = int(partes_chegada[1])
    chegada_numero = horas_chegada + minutos_chegada / 60

    preco_numero = int(preco_texto)

    voo_formatado = (saida_numero, chegada_numero, preco_numero)


    chave_da_rota = (origem, destino)

    if chave_da_rota not in voos:
        voos[chave_da_rota] = []
    voos[chave_da_rota].append(voo_formatado)

print("Dicionário de voos criado com sucesso:")
print(voos)

Dicionário de voos criado com sucesso:
{('FCO', 'LIS'): [(6.316666666666666, 8.216666666666667, 239), (8.066666666666666, 10.983333333333333, 136), (9.516666666666667, 11.716666666666667, 210), (11.116666666666667, 13.4, 171), (12.516666666666667, 14.033333333333333, 234), (14.083333333333334, 15.783333333333333, 226), (15.116666666666667, 17.35, 129), (16.583333333333332, 18.933333333333334, 144), (18.416666666666668, 20.566666666666666, 205), (20.083333333333332, 21.733333333333334, 172)], ('LIS', 'FCO'): [(6.183333333333334, 8.516666666666667, 249), (7.65, 10.4, 219), (9.25, 12.05, 99), (11.133333333333333, 13.116666666666667, 175), (12.3, 14.933333333333334, 172), (13.616666666666667, 15.133333333333333, 250), (15.05, 16.7, 135), (16.85, 19.15, 147), (18.2, 20.283333333333335, 242), (20.083333333333332, 22.1, 261)], ('FCO', 'MAD'): [(6.05, 8.716666666666667, 219), (7.833333333333333, 10.133333333333333, 164), (9.183333333333334, 10.7, 172), (10.55, 13.183333333333334, 132), (12.133

In [ ]:
#voos[('LIS', 'FCO')]
#len(voos[('FCO', 'DUB')])
#print(voos)

In [4]:
def converter_float_para_horario(horario_float):
    horas = int(horario_float)
    minutos = int((horario_float - horas) * 60)
    horario_formatado = f"{horas:02d}:{minutos:02d}"

    return horario_formatado

def imprimir_voos(agenda):
    print('%15s %12s %8s %14s %8s' % ('Cidade (Origem)', 'Voo de Ida', 'Preço', 'Voo de Volta', 'Preço'))
    print('=' * 70)

    total_preco = 0
    for i in range(len(pessoas)): # Usa a variável global 'pessoas'

        cidade_origem = pessoas[i][0]
        codigo_origem = pessoas[i][1]

        indice_ida = agenda[i * 2]
        indice_volta = agenda[i * 2 + 1]

        # Usa as variáveis globais 'voos' e 'destino_final'
        ida = voos[(codigo_origem, destino)][indice_ida]
        volta = voos[(destino, codigo_origem)][indice_volta]

        total_preco += ida[2]
        total_preco += volta[2]

        texto_voo_ida = f"{converter_float_para_horario(ida[0])}-{converter_float_para_horario(ida[1])}"
        texto_voo_volta = f"{converter_float_para_horario(volta[0])}-{converter_float_para_horario(volta[1])}"

        print('%15s %12s %8.2f %14s %8.2f' % (cidade_origem, texto_voo_ida, ida[2],
                                                  texto_voo_volta, volta[2]))

    print('=' * 70)
    print(f'Preço total das passagens: R$ {total_preco:.2f}')

In [5]:
agenda = [0,5, 3,2, 7,1, 1,3, 2,4, 1,9]
imprimir_voos(agenda)

Cidade (Origem)   Voo de Ida    Preço   Voo de Volta    Preço
         Lisboa  06:11-08:31   249.00    14:05-15:46   226.00
         Madrid  11:01-12:39   260.00    09:11-10:41   172.00
          Paris  17:07-20:03   291.00    08:22-11:07   143.00
         Dublin  08:03-10:11    95.00    10:33-12:03    74.00
       Bruxelas  09:07-12:11   364.00    12:20-16:33   500.00
        Londres  08:26-10:45   139.00    19:45-21:45   214.00
Preço total das passagens: R$ 2727.00


In [ ]:
10**12

1000000000000


## Função Objetivo

In [45]:
def funcao_custo(agenda):
    total_preco_passagens = 0
    horarios_chegada_destino = []
    horarios_partida_volta = []

    for i in range(len(pessoas)):
        codigo_origem = pessoas[i][1]

        # Voo de Ida
        indice_voo_ida = agenda[i * 2]
        voo_ida = voos[(codigo_origem, destino)][indice_voo_ida]

        total_preco_passagens += voo_ida[2]
        horarios_chegada_destino.append(voo_ida[1])

        # Voo de Volta
        indice_voo_volta = agenda[i * 2 + 1]
        voo_volta = voos[(destino, codigo_origem)][indice_voo_volta]

        total_preco_passagens += voo_volta[2]
        horarios_partida_volta.append(voo_volta[0])



    horario_ultima_chegada = max(horarios_chegada_destino)
    custo_total_espera_ida = 0
    # Para cada passageiro, calcula quanto tempo ele vai esperar
    for horario_chegada in horarios_chegada_destino:
        tempo_de_espera_horas = horario_ultima_chegada - horario_chegada
        custo_total_espera_ida += tempo_de_espera_horas * custo_espera_por_hora

    horario_primeira_partida = min(horarios_partida_volta)
    custo_total_espera_volta = 0
    # Para cada passageiro, calcula quanto tempo ele vai esperar na volta
    for horario_partida in horarios_partida_volta:
        tempo_de_espera = horario_partida - horario_primeira_partida
        custo_total_espera_volta += tempo_de_espera * custo_espera_por_hora


    custo_final = total_preco_passagens + custo_total_espera_ida + custo_total_espera_volta

    return custo_final

In [81]:
agenda = [1,9, 8,5, 3,5, 2,7, 1,8, 1,1]
custo = funcao_custo(agenda)
print(custo)
agendamento = imprimir_voos(agenda)

3549.166666666667
Cidade (Origem)   Voo de Ida    Preço   Voo de Volta    Preço
         Lisboa  07:39-10:24   219.00    20:04-21:44   172.00
         Madrid  18:48-21:45   246.00    14:18-17:08   190.00
          Paris  11:28-14:39   248.00    14:07-16:08   232.00
         Dublin  09:45-11:50   172.00    17:03-18:03   103.00
       Bruxelas  07:52-11:37   433.00    18:44-22:41   351.00
        Londres  08:26-10:45   139.00    08:18-11:16   122.00
Preço total das passagens: R$ 2627.00


In [73]:
import random as rd
agenda =[]
for i in range(len(pessoas)*2):
  agenda.append(rd.randint(0,9))
print(agenda)
custo_solucao = funcao_custo(agenda)
print(custo_solucao)
imprimir = imprimir_voos(agenda)

[0, 9, 0, 9, 9, 3, 7, 1, 8, 3, 5, 8]
3333.5
Cidade (Origem)   Voo de Ida    Preço   Voo de Volta    Preço
         Lisboa  06:11-08:31   249.00    20:04-21:44   172.00
         Madrid  06:04-08:31   174.00    19:32-21:25   160.00
          Paris  19:52-22:21   173.00    11:07-14:37   262.00
         Dublin  17:11-18:30   108.00    08:22-10:28   149.00
       Bruxelas  18:26-21:29   464.00    10:50-14:16   256.00
        Londres  13:39-15:37   137.00    18:17-21:03   259.00
Preço total das passagens: R$ 2563.00
